<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Imports" data-toc-modified-id="Imports-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Imports</a></span></li><li><span><a href="#Function-definitions" data-toc-modified-id="Function-definitions-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Function definitions</a></span></li><li><span><a href="#Open-Serial-Port" data-toc-modified-id="Open-Serial-Port-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Open Serial Port</a></span><ul class="toc-item"><li><span><a href="#Loop-sending-bytes" data-toc-modified-id="Loop-sending-bytes-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Loop sending bytes</a></span></li><li><span><a href="#Send-bytes-in-HDLC-in-a-loop" data-toc-modified-id="Send-bytes-in-HDLC-in-a-loop-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Send bytes in HDLC in a loop</a></span></li><li><span><a href="#Send-sine-signal-in-a-Loop" data-toc-modified-id="Send-sine-signal-in-a-Loop-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Send sine signal in a Loop</a></span></li></ul></li><li><span><a href="#Open-wav-file" data-toc-modified-id="Open-wav-file-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Open wav file</a></span><ul class="toc-item"><li><span><a href="#Just-play-the-song-direcly-in-Python" data-toc-modified-id="Just-play-the-song-direcly-in-Python-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Just play the song direcly in Python</a></span></li><li><span><a href="#Send-Data-of-song-via-UART-to-PC-and-play-it-in-Python" data-toc-modified-id="Send-Data-of-song-via-UART-to-PC-and-play-it-in-Python-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Send Data of song via UART to PC and play it in Python</a></span></li><li><span><a href="#Send-bytes-of-wav-and-receive-aknowledge" data-toc-modified-id="Send-bytes-of-wav-and-receive-aknowledge-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>Send bytes of wav and receive aknowledge</a></span></li></ul></li><li><span><a href="#Get-decoded-mp3-data" data-toc-modified-id="Get-decoded-mp3-data-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Get decoded mp3 data</a></span></li><li><span><a href="#Check-consistency-of-sent-data" data-toc-modified-id="Check-consistency-of-sent-data-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Check consistency of sent data</a></span></li><li><span><a href="#Debug" data-toc-modified-id="Debug-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Debug</a></span></li></ul></div>

# Imports

In [1]:
import serial
import time
from pathlib import Path
import wave
import pyaudio
import numpy as np
import pandas as pd
import math

# Function definitions

In [36]:
def create_hdlc_frame(data):
    """
    Creates a hdlc frame from bytearray (adds frame delimiters(0x7E) and escapes flags(0x7E, 0x7D) payload)
    
    Parameter:
    ----------
    
    data (bytearray):
        payload data
        
    Returns:
    -------
    
    frame (bytearray):
        payload embedded in hdlc frame
    """
    if type(data) != bytearray:
        msg = "Unexpected data type. Received {}, but expected 'bytearray'".format(type(data))
    
    frame = b'\x7E' + \
            data.replace(b'\x7D', b'\x7D\x5D') \
                .replace(b'\x7E', b'\x7D\x5E') #\
            #+ b'\x7E'
            
    return frame

In [37]:
def get_transmission_chunk(wav, chunk_size):
    frame = create_hdlc_frame(wav.readframes(chunk_size))
    return frame

In [38]:
def comp_send_receive(rec_data, sent_data):
    """
    Compare array rec_data with array sent_data.
    rec_data is expected to consist of n times sent_data, where the first and
    last occurence of sent_data might be incomplete.

    Parameter:
    ----------

    rec_data (np.array):
        Array which contains n-times sent_data,  where the first and
        last occurence of sent_data might be incomplete.
    
    sent_data (np.array):
        Array, which is the "element" of received_array

    Return:
    -------
        True:   rec_data consists exclusivly of several occurences of sent_data
        False:  -"- vice versa
    """
    offs = np.where(sent_data == rec_data[0])[0][0]
    frame_len = len(sent_data)
    len_start_frame = frame_len-offs

    num_recvd_frames, len_end_frame = divmod(len(rec_data)-len_start_frame, frame_len)

    equal = np.array_equal(rec_data[0:len_start_frame], sent_data[offs:])
    for i in range(0, num_recvd_frames):
        equal &= np.array_equal(rec_data[len_start_frame + i*frame_len: len_start_frame +i *frame_len + frame_len], sent_data)
    if len_end_frame != 0:
        equal &= np.array_equal(rec_data[len(rec_data)-len_end_frame:], sent_data[0:len_end_frame])

    return equal

In [39]:
def array_to_big_endian_bytes(array):
    return bytes(array.astype('>i2'))

In [40]:
def create_sine_samples(amplitude, frequency, samplerate):
    """
    Creates an array containing one sine period with defined frequency and amplitude strobed by the samplerate.
    Output is casted to int16
    
    Parameter:
    ----------
    
    amplitude (number):
        desired amplitude
        
    frequency (number):
        desired frequency in Hz
        
    samplerate (number):
        desired frequnecy in Hz
        
    Returns:
    -------
    
    samples (np.array of np.int16):
        mp.array containing sine samples (one period)
    """
    step_size = 2 * math.pi * frequency / samplerate
    samples = [np.int16(amplitude * math.sin(sample)) for sample in np.arange(0, 2*math.pi, step_size)]
    return samples

# Open Serial Port

In [50]:
ser = serial.Serial('COM5', 2e6, timeout=1, parity=serial.PARITY_NONE) # the baudrate of the uart dongle in verry unprecise. 1.5MHz is matched verry good.

In [49]:
ser.close()

## Loop sending bytes

In [15]:
while(True):
    for byte in range(0,0x100,1):
        #print(byte)
        ser.write(bytes([byte])) #, 0x01, 0x7F, 0x3F, 0x0F,0xFF]))
        time.sleep(1e-2)

KeyboardInterrupt: 

In [44]:
ser.write(bytes([0x7E, 0x00, 0x42, 0x7E]))

4

In [10]:
ser.read_all()

b'U\xaaU\xaa'

In [ ]:
while(True):
    time.sleep(1)
    ser.write(bytes([0x00, 0x77, 0x55]))

## Send bytes in HDLC in a loop

In [34]:
data_little_endian = (np.arange(start=0x30, stop=0x40, step=1,dtype=np.int16()))

In [35]:
data_big_endian = array_to_big_endian_bytes(data_little_endian)
hdlc_frame = create_hdlc_frame(data=data_big_endian)

In [36]:
hdlc_frame

b'~\x000\x001\x002\x003\x004\x005\x006\x007\x008\x009\x00:\x00;\x00<\x00=\x00>\x00?~'

In [54]:
while(1):
    ser.write(hdlc_frame)
    ser.read_all()

KeyboardInterrupt: 

In [ ]:
ser.write(hdlc_frame)
ser.read_all()

## Send sine signal in a Loop

In [60]:
sine = create_sine_samples(amplitude=2000//2, frequency=1e3, samplerate=48e3)
sine_len = len(sine)
# get free mem in fpga
response_size = 2
#ser.write(b'\x7E')

ser.reset_input_buffer()
ser.reset_output_buffer()

ser.write(bytes([0x7E]))
time.sleep(1)
rec = ser.read(response_size).hex()
#rec = ser.read(response_size).hex()
if rec == '':
    raise TimeoutError('Timout reached')
free_mem = int(rec, 16)
sine_pos = 0

detect_underflow = False
#read data
while 1:
    print("Free mem:", free_mem)
    print("In buff:", ser.in_waiting)
    if (free_mem < 4000) and (detect_underflow == False):
        detect_underflow = True
    if (free_mem > 4000) and (detect_underflow == True):
        msg = "Potential Underflow!"
        raise serial.SerialException(f'{msg}')
    if free_mem > sine_len:
        # build payload
        start_fragment = sine_len - sine_pos
        remaining_len = free_mem - start_fragment
        frame_cnt, end_fragment = divmod(remaining_len, sine_len)
        payload = sine[sine_pos:] + frame_cnt * sine + sine[0:end_fragment]
        sine_pos = end_fragment

        # serialize frame
        frame = create_hdlc_frame(array_to_big_endian_bytes(np.array(payload)) )       
        
        ser.write(frame)
        time.sleep(1e-3)
        rec = ser.read(response_size).hex()
        if rec == '':
            raise TimeoutError('Timout reached')
        free_mem = int(rec, 16)
    else:
        print('FPGA Buffer full!!')
        time.sleep(5e-3)
        ser.write(b'\x7E')
        time.sleep(1e-3)
        rec = ser.read(response_size).hex()
        if rec == '':
            raise TimeoutError('Timout reached')
        free_mem = int(rec, 16)



Free mem: 4095
In buff: 0
Free mem: 4095
In buff: 0
Free mem: 2177
In buff: 0
Free mem: 185
In buff: 0
Free mem: 3244
In buff: 0
Free mem: 3229
In buff: 0
Free mem: 1709
In buff: 0
Free mem: 200
In buff: 0
Free mem: 3538
In buff: 0
Free mem: 3551
In buff: 0
Free mem: 1886
In buff: 0
Free mem: 198
In buff: 0
Free mem: 3411
In buff: 0
Free mem: 3420
In buff: 0
Free mem: 1818
In buff: 0
Free mem: 218
In buff: 0
Free mem: 3497
In buff: 0
Free mem: 3497
In buff: 0
Free mem: 1878
In buff: 0
Free mem: 173
In buff: 0
Free mem: 3446
In buff: 0
Free mem: 3434
In buff: 0
Free mem: 1819
In buff: 0
Free mem: 142
In buff: 0
Free mem: 3432
In buff: 0
Free mem: 3443
In buff: 0
Free mem: 1837
In buff: 0
Free mem: 218
In buff: 0
Free mem: 3465
In buff: 0
Free mem: 3460
In buff: 0
Free mem: 1874
In buff: 0
Free mem: 214
In buff: 0
Free mem: 3497
In buff: 0
Free mem: 3494
In buff: 0
Free mem: 1863
In buff: 0
Free mem: 233
In buff: 0
Free mem: 3509
In buff: 0
Free mem: 3437
In buff: 0
Free mem: 1761
In buf

Free mem: 3411
In buff: 0
Free mem: 3404
In buff: 0
Free mem: 1832
In buff: 0
Free mem: 231
In buff: 0
Free mem: 3497
In buff: 0
Free mem: 3470
In buff: 0
Free mem: 1809
In buff: 0
Free mem: 155
In buff: 0
Free mem: 3438
In buff: 0
Free mem: 3453
In buff: 0
Free mem: 1787
In buff: 0
Free mem: 146
In buff: 0
Free mem: 3461
In buff: 0
Free mem: 3466
In buff: 0
Free mem: 1833
In buff: 0
Free mem: 142
In buff: 0
Free mem: 3402
In buff: 0
Free mem: 3416
In buff: 0
Free mem: 1851
In buff: 0
Free mem: 253
In buff: 0
Free mem: 3549
In buff: 0
Free mem: 3546
In buff: 0
Free mem: 1896
In buff: 0
Free mem: 239
In buff: 0
Free mem: 3485
In buff: 0
Free mem: 3454
In buff: 0
Free mem: 1801
In buff: 0
Free mem: 164
In buff: 0
Free mem: 3471
In buff: 0
Free mem: 3513
In buff: 0
Free mem: 1815
In buff: 0
Free mem: 172
In buff: 0
Free mem: 3445
In buff: 0
Free mem: 3438
In buff: 0
Free mem: 1775
In buff: 0
Free mem: 172
In buff: 0
Free mem: 3446
In buff: 0
Free mem: 3436
In buff: 0
Free mem: 1773
In buf

In buff: 0
Free mem: 3521
In buff: 0
Free mem: 3487
In buff: 0
Free mem: 1839
In buff: 0
Free mem: 222
In buff: 0
Free mem: 3419
In buff: 0
Free mem: 3408
In buff: 0
Free mem: 1816
In buff: 0
Free mem: 182
In buff: 0
Free mem: 3453
In buff: 0
Free mem: 3388
In buff: 0
Free mem: 1759
In buff: 0
Free mem: 188
In buff: 0
Free mem: 3478
In buff: 0
Free mem: 3500
In buff: 0
Free mem: 1896
In buff: 0
Free mem: 258
In buff: 0
Free mem: 3467
In buff: 0
Free mem: 3420
In buff: 0
Free mem: 1830
In buff: 0
Free mem: 217
In buff: 0
Free mem: 3483
In buff: 0
Free mem: 3472
In buff: 0
Free mem: 1832
In buff: 0
Free mem: 217
In buff: 0
Free mem: 3436
In buff: 0
Free mem: 3428
In buff: 0
Free mem: 1822
In buff: 0
Free mem: 214
In buff: 0
Free mem: 3484
In buff: 0
Free mem: 3518
In buff: 0
Free mem: 1835
In buff: 0
Free mem: 167
In buff: 0
Free mem: 3428
In buff: 0
Free mem: 3428
In buff: 0
Free mem: 1808
In buff: 0
Free mem: 186
In buff: 0
Free mem: 3476
In buff: 0
Free mem: 3448
In buff: 0
Free mem: 

In buff: 0
Free mem: 219
In buff: 0
Free mem: 3496
In buff: 0
Free mem: 3489
In buff: 0
Free mem: 1835
In buff: 0
Free mem: 154
In buff: 0
Free mem: 3405
In buff: 0
Free mem: 3418
In buff: 0
Free mem: 1847
In buff: 0
Free mem: 261
In buff: 0
Free mem: 3531
In buff: 0
Free mem: 3477
In buff: 0
Free mem: 1818
In buff: 0
Free mem: 213
In buff: 0
Free mem: 3490
In buff: 0
Free mem: 3496
In buff: 0
Free mem: 1871
In buff: 0
Free mem: 251
In buff: 0
Free mem: 3513
In buff: 0
Free mem: 3485
In buff: 0
Free mem: 1807
In buff: 0
Free mem: 181
In buff: 0
Free mem: 3475
In buff: 0
Free mem: 3492
In buff: 0
Free mem: 1807
In buff: 0
Free mem: 183
In buff: 0
Free mem: 3461
In buff: 0
Free mem: 3436
In buff: 0
Free mem: 1819
In buff: 0
Free mem: 156
In buff: 0
Free mem: 3434
In buff: 0
Free mem: 3452
In buff: 0
Free mem: 1835
In buff: 0
Free mem: 193
In buff: 0
Free mem: 3455
In buff: 0
Free mem: 3464
In buff: 0
Free mem: 1842
In buff: 0
Free mem: 208
In buff: 0
Free mem: 3452
In buff: 0
Free mem: 3

Free mem: 3414
In buff: 0
Free mem: 1765
In buff: 0
Free mem: 141
In buff: 0
Free mem: 3434
In buff: 0
Free mem: 3437
In buff: 0
Free mem: 1849
In buff: 0
Free mem: 222
In buff: 0
Free mem: 3466
In buff: 0
Free mem: 3464
In buff: 0
Free mem: 1880
In buff: 0
Free mem: 229
In buff: 0
Free mem: 3481
In buff: 0
Free mem: 3496
In buff: 0
Free mem: 1851
In buff: 0
Free mem: 180
In buff: 0
Free mem: 3381
In buff: 0
Free mem: 3400
In buff: 0
Free mem: 1816
In buff: 0
Free mem: 215
In buff: 0
Free mem: 3501
In buff: 0
Free mem: 3496
In buff: 0
Free mem: 1827
In buff: 0
Free mem: 192
In buff: 0
Free mem: 3456
In buff: 0
Free mem: 3472
In buff: 0
Free mem: 1856
In buff: 0
Free mem: 230
In buff: 0
Free mem: 3479
In buff: 0
Free mem: 3455
In buff: 0
Free mem: 1808
In buff: 0
Free mem: 164
In buff: 0
Free mem: 3463
In buff: 0
Free mem: 3499
In buff: 0
Free mem: 1848
In buff: 0
Free mem: 177
In buff: 0
Free mem: 3422
In buff: 0
Free mem: 3409
In buff: 0
Free mem: 1770
In buff: 0
Free mem: 173
In buff

...

Free mem: 3455
In buff: 0
Free mem: 3462
In buff: 0
Free mem: 1771
In buff: 0
Free mem: 131
In buff: 0
Free mem: 3403
In buff: 0
Free mem: 3430
In buff: 0
Free mem: 1812
In buff: 0
Free mem: 199
In buff: 0
Free mem: 3477
In buff: 0
Free mem: 3480
In buff: 0
Free mem: 1836
In buff: 0
Free mem: 224
In buff: 0
Free mem: 3482
In buff: 0
Free mem: 3412
In buff: 0
Free mem: 1762
In buff: 0
Free mem: 168
In buff: 0
Free mem: 3465
In buff: 0
Free mem: 3532
In buff: 0
Free mem: 1924
In buff: 0
Free mem: 237
In buff: 0
Free mem: 3457
In buff: 0
Free mem: 3387
In buff: 0
Free mem: 1806
In buff: 0
Free mem: 233
In buff: 0
Free mem: 3537
In buff: 0
Free mem: 3489
In buff: 0
Free mem: 1843
In buff: 0
Free mem: 213
In buff: 0
Free mem: 3466
In buff: 0
Free mem: 3415
In buff: 0
Free mem: 1824
In buff: 0
Free mem: 219
In buff: 0
Free mem: 3494
In buff: 0
Free mem: 3431
In buff: 0
Free mem: 1782
In buff: 0
Free mem: 161
In buff: 0
Free mem: 3471
In buff: 0
Free mem: 3468
In buff: 0
Free mem: 1826
In buf

Free mem: 3484
In buff: 0
Free mem: 1871
In buff: 0
Free mem: 232
In buff: 0
Free mem: 3461
In buff: 0
Free mem: 3456
In buff: 0
Free mem: 1855
In buff: 0
Free mem: 172
In buff: 0
Free mem: 3359
In buff: 0
Free mem: 3387
In buff: 0
Free mem: 1836
In buff: 0
Free mem: 254
In buff: 0
Free mem: 3504
In buff: 0
Free mem: 3463
In buff: 0
Free mem: 1822
In buff: 0
Free mem: 183
In buff: 0
Free mem: 3406
In buff: 0
Free mem: 3380
In buff: 0
Free mem: 1800
In buff: 0
Free mem: 202
In buff: 0
Free mem: 3490
In buff: 0
Free mem: 3515
In buff: 0
Free mem: 1877
In buff: 0
Free mem: 186
In buff: 0
Free mem: 3447
In buff: 0
Free mem: 3467
In buff: 0
Free mem: 1822
In buff: 0
Free mem: 210
In buff: 0
Free mem: 3476
In buff: 0
Free mem: 3481
In buff: 0
Free mem: 1874
In buff: 0
Free mem: 202
In buff: 0
Free mem: 3431
In buff: 0
Free mem: 3436
In buff: 0
Free mem: 1835
In buff: 0
Free mem: 239
In buff: 0
Free mem: 3478
In buff: 0
Free mem: 3411
In buff: 0
Free mem: 1755
In buff: 0
Free mem: 158
In buff

Free mem: 3448
In buff: 0
Free mem: 1802
In buff: 0
Free mem: 112
In buff: 0
Free mem: 3400
In buff: 0
Free mem: 3462
In buff: 0
Free mem: 1830
In buff: 0
Free mem: 187
In buff: 0
Free mem: 3455
In buff: 0
Free mem: 3475
In buff: 0
Free mem: 1858
In buff: 0
Free mem: 238
In buff: 0
Free mem: 3472
In buff: 0
Free mem: 3442
In buff: 0
Free mem: 1809
In buff: 0
Free mem: 178
In buff: 0
Free mem: 3417
In buff: 0
Free mem: 3454
In buff: 0
Free mem: 1850
In buff: 0
Free mem: 213
In buff: 0
Free mem: 3452
In buff: 0
Free mem: 3461
In buff: 0
Free mem: 1894
In buff: 0
Free mem: 242
In buff: 0
Free mem: 3475
In buff: 0
Free mem: 3449
In buff: 0
Free mem: 1846
In buff: 0
Free mem: 219
In buff: 0
Free mem: 3514
In buff: 0
Free mem: 3492
In buff: 0
Free mem: 1858
In buff: 0
Free mem: 233
In buff: 0
Free mem: 3475
In buff: 0
Free mem: 3439
In buff: 0
Free mem: 1801
In buff: 0
Free mem: 175
In buff: 0
Free mem: 3423
In buff: 0
Free mem: 3472
In buff: 0
Free mem: 1865
In buff: 0
Free mem: 252
In buff

In buff: 0
Free mem: 3497
In buff: 0
Free mem: 3476
In buff: 0
Free mem: 1839
In buff: 0
Free mem: 217
In buff: 0
Free mem: 3473
In buff: 0
Free mem: 3469
In buff: 0
Free mem: 1883
In buff: 0
Free mem: 218
In buff: 0
Free mem: 3476
In buff: 0
Free mem: 3480
In buff: 0
Free mem: 1869
In buff: 0
Free mem: 253
In buff: 0
Free mem: 3432
In buff: 0
Free mem: 3386
In buff: 0
Free mem: 1774
In buff: 0
Free mem: 208
In buff: 0
Free mem: 3497
In buff: 0
Free mem: 3491
In buff: 0
Free mem: 1872
In buff: 0
Free mem: 242
In buff: 0
Free mem: 3469
In buff: 0
Free mem: 3389
In buff: 0
Free mem: 1766
In buff: 0
Free mem: 143
In buff: 0
Free mem: 3432
In buff: 0
Free mem: 3389
In buff: 0
Free mem: 1785
In buff: 0
Free mem: 217
In buff: 0
Free mem: 3524
In buff: 0
Free mem: 3472
In buff: 0
Free mem: 1817
In buff: 0
Free mem: 212
In buff: 0
Free mem: 3443
In buff: 0
Free mem: 3439
In buff: 0
Free mem: 1821
In buff: 0
Free mem: 208
In buff: 0
Free mem: 3474
In buff: 0
Free mem: 3480
In buff: 0
Free mem: 

KeyboardInterrupt: 

In [17]:
ser.in_waiting

0

In [56]:
5 * sine

[0,
 31650,
 16383,
 -23169,
 -28377,
 0,
 31650,
 16383,
 -23169,
 -28377,
 0,
 31650,
 16383,
 -23169,
 -28377,
 0,
 31650,
 16383,
 -23169,
 -28377,
 0,
 31650,
 16383,
 -23169,
 -28377]

# Open wav file

In [7]:
wav_path = Path(r'..\test\Finjark_Master_003.wav')

In [17]:
wav = wave.open(str(wav_path), mode='rb')

In [18]:
wav.close()

## Just play the song direcly in Python

In [13]:
p = pyaudio.PyAudio()  
#open stream  
stream = p.open(format = p.get_format_from_width(wav.getsampwidth()),  
                channels = wav.getnchannels(),  
                rate = wav.getframerate(),  
                output = True)  
#read data  
data = wav.readframes(chunk_size)  

#play stream  
#while data:  
#    stream.write(data)  
#    data = wav.readframes(chunk_size) 

KeyboardInterrupt: 

## Send Data of song via UART to PC and play it in Python

In [85]:
ser.close()
ser = serial.Serial(port='COM5', baudrate=256e3, timeout=5, parity=serial.PARITY_NONE, stopbits=serial.STOPBITS_ONE)

In [114]:
chunk_size = 20
data = None
wav = wave.open(f=str(wav_path / 'Finjark_Master_003.wav'), mode='rb')
p = pyaudio.PyAudio()  
#open stream  
stream = p.open(format = p.get_format_from_width(wav.getsampwidth()),  
                channels = wav.getnchannels(),  
                rate = wav.getframerate(),  
                output = True) 

ser.write(wav.readframes(chunk_size))
#data = ser.read_all()

#read data
#while data:
#    stream.write(data)
#    ser.write(wav.readframes(chunk_size))
#    data = ser.read_all()
    


#play stream  
#while data:  
#    stream.write(data)  
#    data = wav.readframes(chunk_size) 

40

## Send bytes of wav and receive aknowledge

In [46]:
wav_path = Path(r'D:\Sicherungen\Musik_Erwin\Knaat')

In [49]:
free_mem = 0
response_size = 2

wav = wave.open(f=str(wav_path / 'Finjark_Master_003.wav'), mode='rb')
p = pyaudio.PyAudio()  
#open stream  
stream = p.open(format = p.get_format_from_width(wav.getsampwidth()),  
                channels = wav.getnchannels(),  
                rate = wav.getframerate(),  
                output = True) 


ser.write(b'\x7E')
time.sleep(1e-3)
rec = ser.read(response_size).hex()
if rec == '':
    raise TimeoutError('Timout reached')
free_mem = int(rec, 16)
frame = get_transmission_chunk(wav=wav, chunk_size=free_mem // wav.getsampwidth())

#read data
while frame:
    if free_mem > 0:
        ser.write(frame)
        time.sleep(1e-3)
        rec = ser.read(response_size).hex()
        if rec == '':
            raise TimeoutError('Timout reached')
        free_mem = int(rec, 16)
        frame = get_transmission_chunk(wav=wav, chunk_size=free_mem // wav.getsampwidth())
    else:
        print('FPGA Buffer full!!')
        time.sleep(5e-3)
        ser.write(b'\x7E')
        time.sleep(1e-3)
        rec = ser.read(response_size).hex()
        if rec == '':
            raise TimeoutError('Timout reached')
        free_mem = int(rec, 16)

KeyboardInterrupt: 

In [40]:
for byte in range(0,0x100,1):
    ser.write(bytes([byte]))

In [82]:
wav.getnframes()

9879552

# Get decoded mp3 data

# Check consistency of sent data

In [55]:
data = []
with open(r'..\test\debug_data.csv', 'r') as IN:
    for line in IN.readlines():
        data.append(line.replace('\n', '').split(';'))

In [56]:
df = pd.DataFrame(data).T \
                  .rename(columns={0:'channel_0', 1:'channel_1', 2:'channel_2', 3:'channel_3'})

In [61]:
df.head()

,channel_0,channel_1,channel_2,channel_3,4,5
0,60,,51,,,
1,61,None,52,None,None,None
2,62,None,53,None,None,None
3,63,None,54,None,None,None
4,48,None,55,None,None,None


In [75]:
comp_send_receive(df.channel_0.dropna().astype(int).values, np.arange(start=0x30, stop=0x40, step=1))

True

In [77]:
qstr = f"channel_0 == '0'"
df.query(qstr)

,channel_0,channel_1,channel_2,channel_3,4,5


In [73]:
%debug

> <ipython-input-53-8d31feaa4df9>(22)comp_send_receive()
     20         False:  -"- vice versa
     21     """
---> 22     offs = np.where(sent_data == rec_data[0])[0][0]
     23     frame_len = len(sent_data)
     24     len_start_frame = frame_len-offs

ipdb> rec_data
array([51, 52, 53, ..., 59, 60, 61])
ipdb> rec_data[0]
51
ipdb> np.where(sent_data == rec_data[0])
(array([], dtype=int64),)
ipdb> sent_data 
array([12288, 12544, 12800, 13056, 13312, 13568, 13824, 14080, 14336,
       14592, 14848, 15104, 15360, 15616, 15872, 16128])
ipdb> exit


In [20]:
hex(16128)

'0x3f00'

# Debug

In [95]:
a = b'\x7E\x7D'

In [99]:
int(a.hex(), 16)

32381

In [62]:
a.replace(b'\x7D', b'\x7D\x5D').replace(b'\x7E', b'\x7D\x5E').hex()

'7d5e7d5d'

In [55]:
type(wav.readframes(chunk_size))

bytes

In [83]:
ser.timeout

0

In [73]:
(b'\x7E' + wav.readframes(chunk_size).replace(b'\x7D', b'\x7D\x5D').replace(b'\x7E', b'\x7D\x5E') + b'\x7E').hex()

'7e0000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000

In [18]:
ser.close()

In [50]:
wav.

In [96]:
a = np.array([1, 2, 0, 1, 2, 0, 1, 2, 0, 1])
b = np.array([0, 1, 2])
c = np.array([1, 2, 0, 1, 2, 0, 1, 2, 0, 2])

In [99]:
comp_send_receive(rec_data=a, sent_data=b)

True

In [22]:
np.where(c == 1)

(array([0, 3, 6], dtype=int64),)